In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import pandas as pd
import requests
pd.set_option("display.max_columns", 30)

In [2]:
"""
1. get data from S3
2. weather data transformaiton
3. taxi trips transformations - DONE
4. update payment_type - DONE
5. update company_master - DONE
6. update taxi_trips with company and payment_type - DONE
7. upload weather data to S3
8. upload taxi data to S3
9. upload the newest payment_type_master and company_master
"""

'\n1. get data from S3\n2. weather data transformaiton\n3. taxi trips transformations\n4. update payment_type\n5. update company_master\n6. update taxi_trips with company and payment_type\n7. upload weather data to S3\n8. upload taxi data to S3\n9. upload the newest payment_type_master and company_master\n'

Taxi trips transformation codes

In [3]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = (
    f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
    f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
)
response = requests.get(url)

data = response.json()


In [4]:
taxi_trips = pd.DataFrame(data)

In [5]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.info()

taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("H")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23255 entries, 0 to 23254
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     23255 non-null  object
 1   taxi_id                     23255 non-null  object
 2   trip_start_timestamp        23255 non-null  object
 3   trip_end_timestamp          23255 non-null  object
 4   trip_seconds                23252 non-null  object
 5   trip_miles                  23255 non-null  object
 6   pickup_community_area       22726 non-null  object
 7   dropoff_community_area      21440 non-null  object
 8   fare                        23196 non-null  object
 9   tips                        23196 non-null  object
 10  tolls                       23196 non-null  object
 11  extras                      23196 non-null  object
 12  trip_total                  23196 non-null  object
 13  payment_type                23255 non-null  ob

C:\Users\Goga\AppData\Local\Temp\ipykernel_704\1560684747.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("H")


In [6]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,fb8bc1ebde224f8d8cfd0eb4b9d7c5e9a343945a,e2d8418fcdb061eee0a4318fba0a6a1200aaff0143feb0...,2024-06-13T23:45:00.000,2024-06-14T00:00:00.000,504,1.73,32,8,9,1,0,0,10,Mobile,Chicago Independents,41.878865584,-87.625192142,41.899602111,-87.633308037,2024-06-13 23:00:00
1,fb5c3877b9b740e269c2ad9d518186061355120e,5d58f628a7d8340722380e52d042fb43ba69ae9e42559f...,2024-06-13T23:45:00.000,2024-06-13T23:45:00.000,368,6.99,22,10,18.75,0,0,0,18.75,Cash,5 Star Taxi,41.92276062,-87.699155343,41.985015101,-87.804532006,2024-06-13 23:00:00
2,fb2d4e52d97f7b0c6ccc5ee6a52ff59db70fdce6,6544c8051894de482dd3253d911c3e3d713b43beb93de2...,2024-06-13T23:45:00.000,2024-06-14T00:00:00.000,408,4.37,28,28,13,0,0,0,13,Cash,City Service,41.874005383,-87.66351755,41.874005383,-87.66351755,2024-06-13 23:00:00
3,f95b2bc55b544e8fbe1212b9321066675b7086cc,0f831bff43d83f396f2e4950126c6137dcdb60fb4c8580...,2024-06-13T23:45:00.000,2024-06-14T00:15:00.000,1913,19.24,76,31,47.5,13.12,0,4.5,65.62,Credit Card,Sun Taxi,41.980264315,-87.913624596,41.850266366,-87.667569312,2024-06-13 23:00:00
4,f84af0648d5a2b2c0c489af4cf890041d41eed0e,1d8661cf286a18a5105e132056c2aadf0b81a551e440fe...,2024-06-13T23:45:00.000,2024-06-13T23:45:00.000,288,0.5,NaN,NaN,5,3,0,1,9.5,Credit Card,Chicago Independents,NaN,NaN,NaN,NaN,2024-06-13 23:00:00


taxi trips transformation function

In [7]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """ Perform transformation with the taxi data

    Parameters:
    -----------
        taxi_trips : pd.DataFrame
            The dataframe holding the daly taxi trips

    Returns:
    --------
        pd.dataframe
        The cleaned transformed Dataframe holding the taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas datarfame.")

    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
    taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

    taxi_trips.info()

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                            "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("H")

    return taxi_trips

company update codes

In [8]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        "company_id": range(1, len(company_master) + 1),
        "company": company_master
    }
)

company_master.tail()

,company_id,company
30,31,Patriot Taxi Dba Peace Taxi Associat
31,32,3556 - 36214 RC Andrews Cab
32,33,Tac - Blue Diamond Dispatch
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.


In [9]:
new_company_data = [
    {"company": "5167 - 71969 5167 Taxi Inc"},
    {"company": "X"},
    {"company": "Y"},
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

,company
0,5167 - 71969 5167 Taxi Inc
1,X
2,Y


In [10]:
company_max_id = company_master["company_id"].max()
company_max_id

np.int64(35)

In [11]:
new_companies_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_companies_list.append(company)

# one line
# new_companies_list_one_line = [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

new_companies_list

['X', 'Y']

In [12]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
    "company": new_companies_list
})

new_companies_df

,company_id,company
0,36,X
1,37,Y


In [14]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

updated_company_master.tail()


,company_id,company
32,33,Tac - Blue Diamond Dispatch
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.
35,36,X
36,37,Y


In [15]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the company master tih new compaines.

    Parameters:
    -----------
    taxi_trips: pd.DataFrame
        Dataframe holding the dayly taxi trips
    company_master: pd.DataFrame
        Dataframe holding the company_master data

    Returns:
    --------
    pd.DataFrame
        The updated company_master data, if new companies are in the taxi data.
    
    """
    company_max_id = company_master["company_id"].max()
    
    new_companies_list_one_line = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
    new_companies_df = pd.DataFrame({
        "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
        "company": new_companies_list
    })

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [30]:
taxi_trips_company_only = pd.DataFrame({
    "company_id": [1, 2, 3],
    "company": ["5167 - 71969 5167 Taxi Inc", "x", "y"]
})

taxi_trips_company_only

,company_id,company
0,1,5167 - 71969 5167 Taxi Inc
1,2,x
2,3,y


In [31]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [32]:
updated_company_master.tail()

,company_id,company
32,33,Tac - Blue Diamond Dispatch
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.
35,36,X
36,37,Y


payment_type_master codes

In [19]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master) + 1),
        "payment_type": payment_type_master
    }
)


taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id": [1,2,3],
    "payment_type": ["Credit Card", "X", "Y"]
    })

taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [21]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the payment type master with new payment tipes.

    Parameters:
    -----------
    taxi_trips: pd.DataFrame
        Dataframe holding the dayly taxi trips
    payment_type_master: pd.DataFrame
        Dataframe holding the payment_type_master data

    Returns:
    --------
    pd.DataFrame
        The updated payment_type_master data, if new payment tipes are in the taxi data.
    
    """
    payment_type_max_id = payment_type_master["payment_type_id"].max()
    
    new_payment_types_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_types_df = pd.DataFrame({
        "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list) + 1),
        "payment_type": new_payment_types_list
    })

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_types_df], ignore_index=True)

    return updated_payment_type_master

In [22]:
update_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [23]:
update_payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Credit Card
3,4,Unknown
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


Creating a generic update master table function

In [24]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """Extend the master dataframe with new values.

    Parameters:
    -----------
    taxi_trips: pd.DataFrame
        Dataframe holding the dayly taxi trips
    master: pd.DataFrame
        Dataframe holding the master data
    id_column: str
        The Id column of the master dataframe.
    value_column: str
        Name of the column in master_df containing the values.

    Returns:
    --------
    pd.DataFrame
        The updated master data, if new values are in the taxi data.
    
    """
    max_id = master[id_column].max()
    
    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
        value_column: new_values_list
    })

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [27]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column="payment_type_id", value_column="payment_type")

In [28]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Cash
2,3,Credit Card
3,4,Unknown
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


In [33]:
test_company_master = update_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column="company_id", value_column="company")

In [34]:
test_company_master.tail()

,company_id,company
32,33,Tac - Blue Diamond Dispatch
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.
35,36,x
36,37,y


update taxi_trips with the most recenty company_master and payment_type master functions

In [43]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Update taxi_trips with the most recenty company_master and payment_type master codes

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The dataframe with the daly taxi trips.
    payment_type_master : pd.DataFrame
        The payment type master table.
    company_master : pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        Taxi trips data, with only payment_type_id, company_id.
    """
    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")
    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id

In [44]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
10904,083d801ea95f040900ddab6e318dd5ce26d5fe60,599e7935e8f7321862152296420d8552c36d7fe97517f0...,2024-06-13T15:00:00.000,2024-06-13T16:00:00.000,3688,16.61,76,32,44.25,5,0,5,54.75,41.97907082,-87.903039661,41.880994471,-87.632746489,2024-06-13 15:00:00,3,2
20204,4ecf001db2a6fe0f0d325b4b93a8aa925abd3dd6,286e28223cab05d01037fb52fead0f36d84f4b00e159c0...,2024-06-13T08:30:00.000,2024-06-13T08:30:00.000,4,0,43,43,48.75,9.85,0,0,59.1,41.761577908,-87.572781987,41.761577908,-87.572781987,2024-06-13 08:00:00,3,4
6810,ca43217596071d9d1c78cd34b152c2466657f26e,04029395246e38f9476ebd5271accdd06a55086b216c85...,2024-06-13T17:30:00.000,2024-06-13T18:15:00.000,2333,6.08,32,6,22.75,2,0,0,25.25,41.878865584,-87.625192142,41.944226601,-87.655998182,2024-06-13 17:00:00,1,14
10104,3356bbda2f9c170d9f9479b7fec6bd16d59c59a6,a31d2ea87ea4f5a4793c30f84f000b0c6aad4cd956f6ea...,2024-06-13T15:30:00.000,2024-06-13T16:00:00.000,1782,6.4,32,8,22,0,0,0,22,41.880994471,-87.632746489,41.907520075,-87.6266589,2024-06-13 15:00:00,2,7
9234,84bc5a9fd3fcd587621006498ad69912d239108a,f6278bee69dff0de5ec4da1ef85f90c18ce0bf54045fb9...,2024-06-13T16:00:00.000,2024-06-13T17:15:00.000,4440,16.8,32,76,48.25,12.15,0,0,60.4,41.880994471,-87.632746489,41.97907082,-87.903039661,2024-06-13 16:00:00,3,5


weather tansformation function

In [49]:
def transform_weather_data(weather_data) -> pd.DataFrame:
    """Make transformations on the daily weather api response.

    Parameters
    ----------
    weather_data : JSON
        The daily weather data from the Open Meteo API.

    Returns
    -------
    pd.DataFrame
        A dataframe presentation of the data
    """
    weather_data_filtered = {
        "datetime": weather_data["hourly"]["time"],
        "temperature": weather_data["hourly"]["temperature_2m"],
        "wind_speed": weather_data["hourly"]["wind_speed_10m"],
        "rain": weather_data["hourly"]["rain"],
        "precipitation": weather_data["hourly"]["precipitation"]
    }

    weather_df = pd.DataFrame(weather_data_filtered)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])
    # weather_df.head()

    return weather_df

In [50]:
# test

url = "https://archive-api.open-meteo.com/v1/era5"
#date = "2023-06-01"
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")


params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,precipitation,rain"
}

response = requests.get(url, params=params)

weather_data = response.json()

weather_data_df = transform_weather_data(weather_data)


In [52]:
weather_data_df.head()

,datetime,temperature,wind_speed,rain,precipitation
0,2024-06-13 00:00:00,30.8,15.1,0.0,0.0
1,2024-06-13 01:00:00,29.0,13.5,0.0,0.0
2,2024-06-13 02:00:00,27.0,14.2,0.0,0.0
3,2024-06-13 03:00:00,26.0,14.7,0.0,0.0
4,2024-06-13 04:00:00,26.1,23.8,0.0,0.0
